# 09 Case Study

0. Introduction
1. Selecting Topics
    1. Choosing four Topics
    2. Process of Selecting Tweets 
    3. Presenting overview
    4. Topic 1
    5. Topic 2
    6. Topic 3
    7. Topic 4
2. LDA
3. BerTopic

In [2]:
from src.SampleTranslation05.translation_01 import load_samples
import pandas as pd
pd.options.display.max_columns = 30 # default 20
pd.options.display.max_rows = 60 # default 60
pd.options.display.float_format = '{:.2f}'.format
# pd.options.display.precision = 2
from src.explorativ_analysis_05 import split_strings_to_list

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
df_1 = pd.read_excel("samples_cleaned.xlsx")
df_1 = df_1.assign(
    translated_lemmas = df_1['translated_lemmas'].apply(split_strings_to_list),
    lemmas = df_1['lemmas'].apply(split_strings_to_list)
)
df = df_1[~df_1['topics'].isna()]
df = df[df['topics'].isin(['sanctions', 'unjustified_war', 'arms_delivery', 'people_killed'])]
print(df.shape)

(100, 24)


In [8]:
for topic in df['topics'].unique():
    print(df.query(f"topics == '{topic}' ").shape)

(25, 24)
(25, 24)
(25, 24)
(25, 24)


In this case study, i will hand select 100 tweets and divide them into three topics. Then i will run LDA, BertTopic and BertTopic Multilingual on them and evaluate them against my handpicked topics.

In [12]:
from sentence_transformers import SentenceTransformer
model_labse = SentenceTransformer('sentence-transformers/LaBSE')
df['embeddings_lemmas'] = df['lemmas'].str.join(" ").progress_apply(model_labse.encode)
df['embeddings_nouns'] = df['nouns'].str.join(" ").progress_apply(model_labse.encode)
df['cleaned_text_embeddings'] = df['cleaned_text'].str.join(" ").progress_apply(model_labse.encode)


100%|██████████| 100/100 [00:15<00:00,  6.49it/s]


In [23]:
from src.TextEmbeddings08.text_embeddings import visualise_with_dimensional_reduction
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
visualise_with_dimensional_reduction(df,PCA(n_components=2), color_to_show='topics', hover_data=['topics', 'cleaned_text_translated','lang'], row_to_transform='cleaned_text_embeddings')

In [65]:
df.columns

Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'text', 'translated',
       'tweetcreatedts', 'tweetid', 'week', 'cleaned_text_translated',
       'cleaned_text', 'lemmas', 'adjs_verbs', 'nouns', 'entities', 'emojis',
       'translated_lemmas', 'translated_adjs_verbs', 'translated_nouns',
       'translated_entities', 'translated_emojis', 'lang', 'topics',
       'Unnamed: 21', 'topics_robin', 'embeddings_lemmas', 'embeddings_nouns',
       'cleaned_text_embeddings', 'kmeans_cleaned_text_topics'],
      dtype='object')

In [25]:
from sklearn.cluster import KMeans
from bertopic import BERTopic
import numpy as np
cluster_model = KMeans(n_clusters=4)
topic_model = BERTopic(hdbscan_model=cluster_model)

topics, probs = topic_model.fit_transform(df['cleaned_text'].to_list(),embeddings= np.stack(df['cleaned_text_embeddings'].to_list(), axis=0))


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [67]:
df_min['kmeans_cleaned_text_topics']

0      2
1      2
3      1
4      0
5      0
      ..
101    0
102    0
103    3
104    0
105    0
Name: kmeans_cleaned_text_topics, Length: 100, dtype: object

In [66]:
df_min = df[['topics', 'kmeans_cleaned_text_topics','cleaned_text_embeddings','cleaned_text_translated','lang']]

In [73]:
df_1 = df_min.drop('topics',axis=1).rename(columns={'kmeans_cleaned_text_topics':'topics'}).assign(top="k_means")
df_2 = df_min.drop('kmeans_cleaned_text_topics', axis=1).assign(top="robin")

df_comb = pd.concat([df_1,df_2], ignore_index=True)

In [74]:
df_comb

,topics,cleaned_text_embeddings,cleaned_text_translated,lang,top
0,2,"[0.052922703, -0.043487024, 0.0075881383, -0.0...",ukrainewar: the eu sanctions against russia...,de,k_means
1,2,"[0.05280618, -0.052376725, 0.019460771, -0.055...","vogt-wuchter: ""we are shocked by the war in uk...",de,k_means
2,1,"[0.027565662, -0.009013671, 0.0124234315, -0.0...",the interior ministry of ukraine reported on ...,de,k_means
3,0,"[0.02145535, -0.02249327, 0.002416536, -0.0770...",chancellor stop imports from russia immediatel...,de,k_means
4,0,"[0.043950394, -0.035810545, 0.018574186, -0.09...",ukraine everyone complains about high gas pric...,de,k_means
...,...,...,...,...,...
195,people_killed,"[0.05025775, -0.021109823, 0.023870414, -0.077...",ukraine: at least two civilians killed and 10 ...,fr,robin
196,unjustified_war,"[0.043137763, -0.008745956, 0.046960697, -0.06...",alert the international criminal court issues ...,fr,robin
197,unjustified_war,"[0.031750523, -0.034840222, 0.011989078, -0.08...",🔴 ukrainerussianwar urgent‼️ the internationa...,fr,robin
198,unjustified_war,"[0.051095523, -0.033423036, 0.014261467, -0.07...",war in ukraine: russia committed wide range ...,fr,robin


In [75]:
import plotly.express as px

min_embeddings = (PCA(n_components=2).fit_transform(np.array(df_comb['cleaned_text_embeddings'].to_list())))
x = min_embeddings[:,0]
y = min_embeddings[:,1]

x_axis_range = [x.min()-1, x.max()+1]
y_axis_range = [y.min()-1, y.max()+1]

fig = px.scatter(df_comb,
                 y=y, 
                 x=x,
                 range_x=x_axis_range, 
                 range_y=y_axis_range,  hover_data=['cleaned_text_translated','lang'], color='topics', facet_col='top')
fig.update_traces(marker_size=10)
fig.update_layout(showlegend=False)

fig.show()